## Version Description:
 * In this version - we predict just the __MTR Hole Diameter__ (`R600_HD`)in TZ6 using [Bayesian Neural Network with MC DropOut](https://github.com/valyome/Neural-Networks-with-MC-Dropout/). 
 * Artificial data points synthesised from SMOTE are used to train the ML model. 
 * The input features are `TZ6_Flow`,`MIXP`,`AMBP`,`AMBT` and the TZ6 CLR Restrictors.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np

#Set some numpy print options for displaying numpy arrays to fit maximum width of cell
np.set_printoptions(precision=3, edgeitems=30, linewidth=1000,formatter=dict(float=lambda x: "%.6g" % x)) 
from functools import reduce

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
plt.style.use('seaborn-whitegrid')

### Loading Data

In [4]:
input_features = ['TZ_Flow','MIXP','AMBP','AMBT','R610_HS1','R611_HS1','R612_HS1','R613_HS1']
usecols = ['HoV']+input_features+['R600_HD']

In [5]:
LTR_df = pd.read_csv('../../data/LTR_dataset.csv', usecols = usecols)[usecols]
LTR_df.head()

,HoV,TZ_Flow,MIXP,AMBP,AMBT,R610_HS1,R611_HS1,R612_HS1,R613_HS1,R600_HD
0,A1,887.134249,2600.0000,101401.6000,299.386667,131,136,120,120,148
1,A2,886.764050,2600.0000,101576.3000,298.448667,131,136,120,120,149
2,A3,926.224856,2606.1928,102136.6035,297.109024,131,136,114,120,152
3,A4,915.139474,2599.8998,103195.6642,295.060027,131,136,120,120,154
4,A5,891.635528,2600.0000,102856.2000,294.755833,145,153,130,130,148


In [6]:
ASD_df = pd.read_csv('../../data/ASD_dataset.csv', usecols = usecols)[usecols]
ASD_df.head()

,HoV,TZ_Flow,MIXP,AMBP,AMBT,R610_HS1,R611_HS1,R612_HS1,R613_HS1,R600_HD
0,ASD0,887.843325,2696.2864,101444.530,299.057324,131,136,120,120,148
1,ASD1,887.477250,2615.7850,101391.560,298.509995,131,136,120,120,148
2,ASD2,887.476875,2648.8844,101513.720,298.961758,131,136,120,120,148
3,ASD3,886.590375,2592.1955,101393.790,298.661206,131,136,120,120,148
4,ASD4,888.331625,2654.4529,101460.547,299.539189,131,136,120,120,148


In [7]:
ASD_df['R600_HD'].value_counts()

154    250
160    200
155    200
153    200
152    200
151    200
149    200
146    200
148    195
150    150
Name: R600_HD, dtype: int64

### Train BNN with FDDN (Simulation) data:

In [8]:
# Extract pandas dataframe values to numpy array
train_data = ASD_df[input_features+['R600_HD']].astype(np.float).values
test_data  = LTR_df[input_features+['R600_HD']].astype(np.float).values
print("Number of training samples:", len(train_data))
print("\nNumber of testing samples:", len(test_data))

# Separate arrays into input and output components (predictors and response variables)
x_train, y_train  = train_data[:,:-1] ,train_data[:,-1:]
x_test, y_test = test_data[:,:-1] ,test_data[:,-1:]

Number of training samples: 1995

Number of testing samples: 34


In [9]:
print('X_train Shape:',x_train.shape,'Y_Train Shape:', y_train.shape)
print('X_test Shape:',x_test.shape,'Y_Test Shape:', y_test.shape)

X_train Shape: (1995, 8) Y_Train Shape: (1995, 1)
X_test Shape: (34, 8) Y_Test Shape: (34, 1)


### Bayesian Neural Network using MC DropOut

In [10]:
import BNN_MonteCarlo_Dropout

Using TensorFlow backend.


In [11]:
# Train the BNN
bnn = BNN_MonteCarlo_Dropout.net(x_train, y_train, n_epochs=2000, n_hidden=[24,12], normalize=True)

In [12]:
# Test on ~25% of the data.
rmse, Yt_hat, MC_pred, predictive_variance = bnn.predict(x_test, y_test)

34/34 [==============================] - 0s 2ms/step
Standard rmse 0.427146
MC rmse 0.413251
test_ll -5.090435


In [13]:
print('Shape of Predictions:',Yt_hat.shape)

Shape of Predictions: (1000, 1, 34, 1)


In [14]:
Yt_hat.shape[2]

34

In [15]:
y_preds = np.zeros((Yt_hat.shape[2], Yt_hat.shape[0])) # empty array to be populated
y_means = [] # save mean for each predicted point
y_std=[] # save standard dev for each predicted
y_mins = [] 
y_maxes = []

for j in range(Yt_hat.shape[2]):
    for i in range(Yt_hat.shape[0]):
        y_preds[j][i] = Yt_hat[i][0][j] # convert the predictions into a more readable format
    y_means.append(y_preds[j].mean()) # get the mean for each prediction
    y_std.append(y_preds[j].std()) # get the standard deviation
    y_mins.append(np.amin(y_preds[j],axis = 0)) # get the min value in the array
    y_maxes.append(np.amax(y_preds[j],axis = 0)) # get the min value in the array

In [16]:
print('Length of Y_Predictions:',len(y_preds))
print('Nr. of Predictions per test point:',len(y_preds[0]))

Length of Y_Predictions: 34
Nr. of Predictions per test point: 1000


In [17]:
print(y_means)

[149.03831114196777, 149.03533010864257, 152.1414044647217, 153.6650467376709, 147.89065802001954, 147.93859181213378, 148.08027896118165, 152.9853837738037, 151.9724754486084, 149.98781016540528, 151.91050390625, 153.79195622253417, 150.82058584594728, 148.38896243286132, 148.33323623657228, 149.40951197814942, 146.46499026489258, 146.362189743042, 148.22337844848633, 148.08859355163574, 151.8567837677002, 148.07824252319335, 153.77082160949706, 153.95286401367187, 154.0314150085449, 148.17631088256837, 148.7063896484375, 154.81099784851074, 148.56503089904785, 149.904157623291, 150.29372967529298, 159.4950488128662, 147.8306420135498, 148.69678100585938]


In [18]:
print(y_std)

[0.5301389546301433, 0.5083348009226839, 0.5712331230225096, 0.8105150975222901, 0.6839327150598437, 0.5537954844179578, 0.577478704770383, 0.4020934817324604, 0.35349254888035897, 0.6435528268962424, 0.378562508515385, 0.45376828348777326, 0.10226932996057041, 0.7218650347842221, 0.6429319668504203, 0.45433827457301407, 1.3404388265792533, 0.9235614455648211, 0.9223681310186291, 0.9924463703702914, 0.336604960291327, 0.6660240933386797, 0.9101214698382107, 0.4958052262496131, 0.917009751251136, 1.0072719666942194, 0.5348437761119218, 0.5320695408989952, 0.578767267067843, 0.316741358455785, 0.5464160876329575, 1.0576157725602569, 0.7833630978515799, 0.5975444008605487]


In [19]:
print(y_mins)

[148.1375274658203, 148.1639404296875, 150.8431854248047, 150.35140991210938, 145.45294189453125, 147.50942993164062, 147.4407501220703, 151.49266052246094, 150.3904571533203, 148.07901000976562, 150.38034057617188, 151.37979125976562, 149.92007446289062, 147.0724639892578, 147.9278106689453, 149.0260009765625, 145.31243896484375, 145.6858367919922, 147.52989196777344, 145.4483184814453, 150.2334442138672, 147.42117309570312, 150.6992645263672, 151.56320190429688, 149.81329345703125, 143.758056640625, 148.1434783935547, 150.6898651123047, 147.93765258789062, 149.20425415039062, 149.60479736328125, 154.5840606689453, 146.53074645996094, 148.26441955566406]


In [20]:
print(y_maxes)

[151.1905975341797, 151.1243438720703, 155.7587432861328, 155.18760681152344, 152.34715270996094, 150.65049743652344, 151.15634155273438, 154.74720764160156, 153.12661743164062, 152.7500457763672, 154.1722869873047, 156.9951934814453, 151.46253967285156, 151.52755737304688, 151.423095703125, 151.77105712890625, 152.22840881347656, 150.91265869140625, 155.57383728027344, 161.35391235351562, 152.8231964111328, 151.0707244873047, 158.22406005859375, 155.60243225097656, 155.71617126464844, 154.0477752685547, 151.00779724121094, 155.55661010742188, 151.4568328857422, 151.72158813476562, 153.35128784179688, 160.6728057861328, 151.1573944091797, 151.9889373779297]


In [21]:
y_true = LTR_df['R600_HD'].values
HoV_test_data = LTR_df['HoV'].values

In [22]:
print('No. of Test HoVs:',len(HoV_test_data),'\nHoVs of Test Points:',HoV_test_data)
print('Test Points-Org Y:', y_true)

No. of Test HoVs: 34 
HoVs of Test Points: ['A1' 'A2' 'A3' 'A4' 'A5' 'A6' 'C1' 'C2' 'C3' 'C4' 'C5' 'C6' 'D1' 'D2' 'E1' 'E2' 'F1' 'F2' 'F3' 'H1' 'I1' 'J1' 'M1' 'M2' 'P1' 'Q1' 'Q2' 'S1' 'S2' 'S3' 'S4' 'S5' 'T1' 'T2']
Test Points-Org Y: [148 149 152 154 148 148 148 153 152 150 152 154 151 148 148 148 146 146 148 148 152 148 154 154 154 148 148 155 148 150 150 160 148 148]


### Plot BNN Predictions with Mean and Error Bars

In [23]:
from bokeh.models import HoverTool,ColumnDataSource,Label,Range1d
from bokeh.plotting import figure, show, output_file,output_notebook
from bokeh.models.markers import CircleX
output_notebook()

Loading BokehJS ...

In [24]:
# create the coordinates for the errorbars
y_xs = []
ys_std = []
y_ys = []

for x, y_mean,std,min_dia, max_dia in zip(HoV_test_data,y_means,y_std,y_mins,y_maxes):
    y_xs.append((x, x))
    ys_std.append((y_mean - std, y_mean + std))
    y_ys.append((y_mean - (y_mean - min_dia), y_mean + (max_dia - y_mean)))

In [25]:
# plot the points
f = figure(x_range=HoV_test_data, title='BNN MC DropOut - Dia Predictions with Confidence Interval', width=1400)
dot_hover_tooltips = HoverTool(names=['DiaPoints'],tooltips=[("HoV","@x"),("Dia", "@y mm")])
whisker_hover_tooltips = HoverTool(names=['Whiskers'],tooltips=[("Dia", "@y mm")])
multiline_hover_tooltips = HoverTool(names=['LinePlot'],tooltips=[("Dia", "$y mm")])

f.add_tools(dot_hover_tooltips,whisker_hover_tooltips,multiline_hover_tooltips)

f.xaxis.axis_label = 'Head of Versions'
f.yaxis.axis_label = 'Diameter Predictions (mm)'

f.multi_line(y_xs, ys_std, color='blue', line_width = 3, legend = 'Mean Prediction +/- Std.Dev')
f.multi_line(y_xs, y_ys, color='deepskyblue',name='LinePlot',legend = 'Mean Prediction +/- Min & Max Value')
f.diamond(HoV_test_data, y_true, color='red', size=12, line_alpha=0,name='DiaPoints', legend = 'True Dia')
f.circle(HoV_test_data, y_means, color='blue', size=9, line_alpha=0,name='DiaPoints', legend = 'Predicted Dia (Mean)')

# whiskers (almost-0 height rects simpler than segments)
for i in range(len(y_xs)):
    f.rect(y_xs[i], y_ys[i], 0.2, 0.001, line_color="deepskyblue",name='Whiskers')

# f.y_range=Range1d(135, 165)
f.legend.location = "top_center"
f.legend.click_policy="hide"

# output_file('plots/BNN_MCDropOut_v3_ASD_LTR_Predictions.html', mode='inline')

In [26]:
show(f)